### Validation
Validating a credit card involves the Luhn algorithm works as follows:
1. The Luhn algorithm starts from the last digit which is called the check digit. Then moving left from this check digit (←), double the value of every digit at even indices.
2. If the result of this doubling operation is greater than 9 (e.g., 6 × 2 = 12), then subtract 9 from the result (e.g., 12: 12 − 9 = 3) or, equivalently, add the digits of the result (e.g., 12: 1 + 2 =3).
3. Now sum all the digits (including the check digit).
4. If the total is divisible by 10 then the number is valid; otherwise, it is not valid.

In [1]:
# Validate the card the old fashioned way, function accepts 1 argument: number, returns true/false plus issuer

def validate(number: str):
    card_number_lst = [int(num) for num in number]
    check_digit = card_number_lst.pop(-1)
    card_number_lst.reverse()
    card_number_lst = [num * 2 if i % 2 == 0 else num for i, num in enumerate(card_number_lst)]
    card_number_lst = [num - 9 if num > 9 else num for num in card_number_lst]
    total = sum(card_number_lst) + check_digit
    return total % 10 == 0
validate('345106858850943')

True

### Card Type
Only looking to determine if the number belongs to these 4 issuers: Visa, MasterCard, American Express, Discover
A card from a given issuer always starts with this number:
- Visa: 4
- MasterCard: 51, 52, 53, 54, 55
- American Express: 34, 37
- Discover: 6011

In [2]:
def card_type(number: str):
    if number[0] == '4':
        return 'Visa'
    elif number[:2] in ['51', '52', '53', '54', '55']:
        return 'MasterCard'
    elif number[:2] in ['34', '37']:
        return 'American Express'
    elif number[:4] == '6011':
        return 'Discover'
    else:
        return 'Unknown'

card_type('345106858850943')

'American Express'

### Generation
To make this efficient and mitigate the need to post my own CC numbers on the internet, I have written the following function that generates a valid credit card number for a given issuer.

In [3]:
import random
visa = ['4']
mastercard = ['51', '52', '53', '54', '55']
discover = ['6011']
amex = ['34', '37']
def generate(issuer: str) -> str:
    if issuer == 'visa':
        return generate_credit_card(visa[0], 16)
    elif issuer == 'mastercard':
        return generate_credit_card(random.choice(mastercard),16)
    elif issuer == 'discover':
        return generate_credit_card(discover[0],16)
    elif issuer == 'amex':
        return generate_credit_card(random.choice(amex),15)
    else:
        return 'Unknown'


def generate_credit_card(prefix: str = '', length: int = 16) -> str:
    if length < 2:
        raise ValueError("Length must be at least 2")
    if len(prefix) > 4:
        raise ValueError("Prefix length must be at most 4")

    # Generate the random part of the credit card number
    random_part_length = length - len(prefix)
    random_part = ''.join(str(random.randint(0, 9)) for _ in range(random_part_length - 1))

    # Combine prefix and random part
    card_number = prefix + random_part
    digits = [int(digit) for digit in card_number]
    odd_digits = digits[-1::-2]
    even_digits = digits[-2::-2]
    checksum = sum(even_digits) + sum(sum(divmod(2 * digit, 10)) for digit in odd_digits)
    check_digit = 9 * checksum % 10

    # Append the check digit to the credit card number
    card_number += str(check_digit)

    return card_number    
    
[generate('visa'),
generate('mastercard'),
generate('discover'),
generate('amex')]

['4694846153892921', '5494351518032007', '6011065161962581', '375571449414991']

In [4]:
# Put it all together
numbers = [generate('visa'),
 generate('mastercard'),
 generate('discover'),
 generate('amex')]
for number in numbers:
    print(f'{number} is a {card_type(number)} card and is {validate(number)}')

4665182338717691 is a Visa card and is True
5443929232035968 is a MasterCard card and is True
6011276361205743 is a Discover card and is True
371832681641426 is a American Express card and is True


### Editorial
This is very easy code to write (I swiped all of it from the web), and it executes very quickly. It is obviously more efficient and simpler than using an LLM for the purpose of validating credit card numbers, however, it requires that someone know python (or coding in general). The directions for verifying a credit card number have been expressed in plain english above, and a person could be given those instructions and verify credit card numbers, no python necessary. A person with no technical expertise could also give these guidelines to an LLM and ask the LLM to verify credit card numbers. There are products coming onto the market that abstract away all the implementation and deployment steps of setting up an LLM to do something, and allow you to create processes out of LLM blocks, passing the output of one in as the input to the next. Rather than each block executing computer code, each block queries an LLM with instructions that can be given in human language. This empowers the non-technical to build functions that can automate tasks. We are going to explore whether an LLM can capably validate CC numbers given instructions in english. We will also see if an LLM can generate a valid CC number given instructions in english.

### Validating with an LLM
#### ChatGPT
As an experiment I tried to get ChatGPT to do this. This was the initial prompt I gave it:
```
you are a function machine. Your purpose is to return true if a credit card number is valid. The following is how you would determine if a credit card number is valid:
Validating a credit card involves the Luhn algorithm works as follows:
1. The Luhn algorithm starts from the last digit which is called the check digit. Then moving left from this check digit (←), double the value of every digit at even indices.
2. If the result of this doubling operation is greater than 9 (e.g., 6 × 2 = 12), then subtract 9 from the result (e.g., 12: 12 − 9 = 3) or, equivalently, add the digits of the result (e.g., 12: 1 + 2 =3).
3. Now sum all the digits (including the check digit).
4. If the total is divisible by 10 then the number is valid; otherwise, it is not valid.


is the following number a valid credit card number?
345106858850943
```
That is a valid credit card number so it should have replied true, instead it replied false and gave this explanation of its thoughts.
```
To determine if the credit card number 345106858850943 is valid using the Luhn algorithm, we need to follow these steps:

Starting from the right (the check digit), double the value of every other digit.
If the doubled value is greater than 9, subtract 9 from it.
Sum all the digits.
If the total sum is divisible by 10, the credit card number is valid.
Let's apply these steps:

Reverse the number: 349058858601543
Double every other digit starting from the second digit from the right (the check digit is not doubled):
3(2) 4 9(2) 0 5(2) 8 8(2) 6 0(2) 1 5(2) 4 3

After doubling and, if necessary, subtracting 9:
3(6) 4 9(4) 0 5(10) 8 8(16) 6 0(2) 1 5(10) 4 3

Sum all the digits:
3 + 6 + 4 + 9 + 4 + 0 + 5 + 1 + 0 + 8 + 8 + 1 + 6 + 6 + 0 + 2 + 1 + 5 + 1 + 0 + 4 + 3 = 82

Check if the sum is divisible by 10:
82 % 10 = 2 (not divisible by 10)

Since the sum is not divisible by 10, the credit card number 345106858850943 is not valid according to the Luhn algorithm.
```
The mistake was that it forgot to remove the checksum digit before reversing the string. After going back and forth I got it to do the process correctly, however, when I gave it another credit card number, it forgot the corrections I had already given it and made the same mistakes again.
### Purpose built LLM

In [12]:
from langchain_community.llms import Ollama

llm = Ollama(model="llama2")

llm.invoke("Take a deep breath before answering. The following is how you would determine if a credit card number is valid:\nValidating a credit card involves the Luhn algorithm works as follows:\n1. The Luhn algorithm starts from the last digit which is called the check digit. Then moving left from this check digit (←), double the value of every digit at even indices.\n2. If the result of this doubling operation is greater than 9 (e.g., 6 × 2 = 12), then subtract 9 from the result (e.g., 12: 12 − 9 = 3) or, equivalently, add the digits of the result (e.g., 12: 1 + 2 =3).\n3. Now sum all the digits (including the check digit).\n4. If the total is divisible by 10 then the number is valid; otherwise, it is not valid.\n\n\nis the following number a valid credit card number?\n345106858850943")

'To validate the credit card number `345106858850943` using the Luhn algorithm, we will follow the steps mentioned in the problem statement:\n\n1. Start with the last digit of the number, which is the check digit `8`.\n2. Double the value of every even-indexed digit (i.e., `0`, `2`, `4`, `6`, and `8`). The results are:\n\t* `0 → 0 × 2 = 0`\n\t* `2 → 2 × 2 = 4`\n\t* `4 → 4 × 2 = 8`\n\t* `6 → 6 × 2 = 12`\n\t* `8 → 8 × 2 = 16`\n3. If the result of the doubling operation is greater than 9, subtract 9 from the result (or add the digits of the result). The results are:\n\t* `16 → 16 - 9 = 7`\n4. Now, sum all the digits (including the check digit) and check if the total is divisible by 10. In this case, the sum is:\n\t* `3 + 4 + 5 + 1 + 0 + 8 + 5 + 9 = 34`\n\nSince the total is not divisible by 10, the credit card number `345106858850943` is not a valid credit card number.'

In [13]:
from langchain_community.llms import Ollama

llm = Ollama(model="llama2:13b")

llm.invoke("Take a deep breath before answering. The following is how you would determine if a credit card number is valid:\nValidating a credit card involves the Luhn algorithm works as follows:\n1. The Luhn algorithm starts from the last digit which is called the check digit. Then moving left from this check digit (←), double the value of every digit at even indices.\n2. If the result of this doubling operation is greater than 9 (e.g., 6 × 2 = 12), then subtract 9 from the result (e.g., 12: 12 − 9 = 3) or, equivalently, add the digits of the result (e.g., 12: 1 + 2 =3).\n3. Now sum all the digits (including the check digit).\n4. If the total is divisible by 10 then the number is valid; otherwise, it is not valid.\n\n\nis the following number a valid credit card number?\n345106858850943")

"\nOkay, let's take a deep breath and go through the process of validating this credit card number:\n\n1. The last digit is 943.\n2. Starting from the last digit, we double every digit at even indices:\n\n6 (doubled) = 12\n8 (doubled) = 16\n5 (doubled) = 10\n1 (doubled) = 2\n\nSo, the result of doubling the digits is:\n\n12 + 16 + 10 + 2 = 40\n\n3. Since the result is greater than 9, we subtract 9 from it:\n\n40 - 9 = 31\n\n4. Now, we sum all the digits, including the check digit:\n\n3 + 4 + 5 + 6 + 8 + 9 + 4 + 3 = 37\n\n5. Since the total is not divisible by 10, this credit card number is not valid.\n\nTherefore, the answer is no, the given credit card number is not a valid credit card number."

In [14]:
from langchain_community.llms import Ollama

llm = Ollama(model="wizard-math:13b")

llm.invoke("Take a deep breath before answering. The following is how you would determine if a credit card number is valid:\nValidating a credit card involves the Luhn algorithm works as follows:\n1. The Luhn algorithm starts from the last digit which is called the check digit. Then moving left from this check digit (←), double the value of every digit at even indices.\n2. If the result of this doubling operation is greater than 9 (e.g., 6 × 2 = 12), then subtract 9 from the result (e.g., 12: 12 − 9 = 3) or, equivalently, add the digits of the result (e.g., 12: 1 + 2 =3).\n3. Now sum all the digits (including the check digit).\n4. If the total is divisible by 10 then the number is valid; otherwise, it is not valid.\n\n\nis the following number a valid credit card number?\n345106858850943")

'\nStep 1: Identify the check digit\nThe check digit is the last digit of the credit card number, which in this case is 9.\n\nStep 2: Double the even-indexed digits\nWe will double the following digits: 3, 4, 5, and 8 (all even-indexed digits).\n3 * 2 = 6\n4 * 2 = 8\n5 * 2 = 10\n8 * 2 = 16\n\nStep 3: Apply the modulo 9 rule\nIf the result is greater than 9, we need to subtract 9 until we get a number between 0 and 9.\n6 % 9 = 6\n8 % 9 = 8\n10 % 9 = 1\n16 % 9 = 7\n\nStep 4: Add the results from Step 3\nNow, we will add all the digits we got in Step 3:\n6 + 8 + 1 + 7 = 22\n\nStep 5: Check if the total is divisible by 10\nThe total number is 22, which is not divisible by 10.\n\nConclusion: The credit card number is not valid.\n\nThe answer is: The algorithm works as follows:\n1. Start with the last digit (the check digit) and double every other digit.\n2. If the result is greater than 9, subtract 9 from the result.\n3. Add up all the digits.\n4. If the total is divisible by 10, the number

In [15]:
from langchain_community.llms import Ollama

llm = Ollama(model="wizard-math")

llm.invoke("Take a deep breath before answering. The following is how you would determine if a credit card number is valid:\nValidating a credit card involves the Luhn algorithm works as follows:\n1. The Luhn algorithm starts from the last digit which is called the check digit. Then moving left from this check digit (←), double the value of every digit at even indices.\n2. If the result of this doubling operation is greater than 9 (e.g., 6 × 2 = 12), then subtract 9 from the result (e.g., 12: 12 − 9 = 3) or, equivalently, add the digits of the result (e.g., 12: 1 + 2 =3).\n3. Now sum all the digits (including the check digit).\n4. If the total is divisible by 10 then the number is valid; otherwise, it is not valid.\n\n\nis the following number a valid credit card number?\n345106858850943")

'\nStep 1: Identify the check digit and the rest of the digits.\nThe check digit is the last digit, which in this case is 3. The remaining digits are 3451068588509.\n\nStep 2: Double every digit at even indices (starting from the second last digit).\nWe will now double every even-indexed digit and apply the rule mentioned in the algorithm for numbers greater than 9.\n\nOriginal number: 345106858850943\nDoubled even-indexed digits: 7 5 0 0 9 8 0 0 2\n\nStep 3: Sum all the digits (including the check digit).\nNow, we will add all the digits together:\n\n7 + 5 + 1 + 0 + 0 + 0 + 9 + 8 + 0 + 0 + 2 + 3 = 46\n\nStep 4: Check if the total is divisible by 10.\nFinally, we will check if the sum (46) is divisible by 10. In this case, it is not (46 ÷ 10 = 4 remainder 6).\n\nConclusion: Since the sum of the digits (46) is not divisible by 10, the credit card number 345106858850943 is not valid.\n\nThe answer is: Since the sum of the digits is not divisible by 10, the credit card number is not valid